In [2]:
from pathlib import Path
import pandas as pd

In [9]:
mbdu_bids_root = Path('/data/MBDU/ABCD/BIDS/NKI_script/MID')
dsst_bids_root = Path('/data/ABCD_DSST/bids_20190215/')
mbdu_raw_root = Path('/data/MBDU/ABCDraw')
mriqc_outdir = Path('/data/ABCD_DSST/bids_20190215/derivatives/mriqc')

rsync_script = Path('raw_rsync.sh')

In [4]:
dsst_subs = sorted(list(dsst_bids_root.glob('sub-*')))
dsst_pls = set([ss.parts[-1][4:] for ss in dsst_subs])
dsst_subs = set([ss.parts[-1] for ss in dsst_subs])

In [10]:
# Get a list of all the jsons that have been produced by MRIQC
jsons = sorted(list(mriqc_outdir.glob('**/*.json')))

json_df = []
for sub_json in jsons:
    row = {}
    row['scan_name'] = sub_json.parts[-1].split('.')[0]
    row['path'] = sub_json
    json_df.append(row)
json_df = pd.DataFrame(json_df)

In [12]:
json_df['pl'] = json_df.scan_name.str.split('_').str[0].str.split('-').str[-1]

In [5]:
mbdu_raw_subs = sorted(list((mbdu_raw_root / 'T1').glob('*.tgz')))

In [6]:
mbdu_raw_pls = set([ss.parts[-1].split('_')[0] for ss in mbdu_raw_subs])

In [7]:
no_bids = mbdu_raw_pls.difference(dsst_pls)

In [26]:
cmds = ['#! /bin/bash']
for nb in no_bids:
    cmds.append(f'rsync -achR {mbdu_raw_root}/./*/{nb}*.tgz abcdqc_backend_user:/abcdqc_data/ABCDraw/')
for nb in json_df.pl.values[:100]:
    cmds.append(f'rsync -achR {mbdu_raw_root}/./*/{nb}*.tgz abcdqc_backend_user:/abcdqc_data/ABCDraw/')
cmds.append('')

In [27]:
rsync_script.write_text('\n'.join(cmds))

33823